In [1]:
import json
from py2neo import Node, Relationship, Graph, NodeMatcher, RelationshipMatcher

In [2]:
graph = Graph("neo4j://10.130.32.3:7687", auth=("neo4j", "12345678"))
node_matcher = NodeMatcher(graph)

In [3]:
with open("../res/苗药方剂学_pre3.json","r") as file:
    data = json.load(file)

In [4]:
def clean_db():
    pt = node_matcher.match().all()
    for i in pt:
        graph.delete(i)

In [5]:
def new_node(parent, child:Node):
    graph.create(parent)
    graph.create(Relationship(parent, "用方", child))

In [6]:
def new_child(parent, child:Node):
    graph.create(parent)
    graph.create(Relationship(parent, "属于", child))

In [7]:
def set_node(parent:Node, nodes:list[dict]):
    for node in nodes:
        try:
            child = Node("Node", **node)
        except:
            child = Node("Node", node)
        new_node(parent, child)

In [8]:
def set_title(parent:Node, child_data:dict, name:str):
    child_names = child_data.get("child",None)
    title = Node("Title",name=name, 信息=";".join(child_data.get("title",list())),alias=",".join(child_data.get("alias",list())))
    new_child(parent, title)
    if child_names is not None:
        for child_name in child_names:
            set_title(title, child_data[child_name],child_name)
    else:
        nodes = child_data.get("data",None)
        if nodes is not None:
            set_node(title, nodes)
        else:
            return None

In [9]:
def main():
    clean_db()
    book = Node("Book", name="苗药方剂")
    graph.create(book)
    child_names = data["child"]
    for child_name in child_names:
        set_title(book, data[child_name],child_name)

In [11]:
main()

In [ ]:
clean_db()

In [81]:
pt = node_matcher.match().all()

In [ ]:
pt